In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install wandb
!pip install imbalanced-learn
!pip install delayed


In [2]:
#!pip install imbalanced-learn
#!pip install delayed

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
import sklearn
from imblearn.over_sampling import RandomOverSampler

In [3]:
# path of dataset
data_path = "./drive/MyDrive/Colab Notebooks/data/Room Detection Dataset/Training Dataset/"
data_arrays_path = "./drive/MyDrive/Colab Notebooks/data_arrays/Room Detection Dataset/Training Dataset/"

# assigning parameters for MFCC feature extraction
max_len_nfft = 597
n_mfcc = 128

In [4]:
# Saving data to array file
save_data_to_array(data_path=data_path, data_arrays_path=data_arrays_path, max_len=max_len_nfft, n_mfcc=n_mfcc)

Saving vectors of label - 'Washroom': 100%|██████████| 500/500 [00:20<00:00, 24.69it/s]
Saving vectors of label - 'RoomDetection Negative': 100%|██████████| 567/567 [00:20<00:00, 28.01it/s]


38
38
41
47
47
48
49
50
51
52
54
55
55
59
59
60
61
61
62
64
65
65
67
67
68
68
68
69
69
69
69
69
70
70
70
70
70
70
71
72
72
72
73
73
73
73
73
74
74
74
74
74
74
74
74
75
75
76
76
76
76
76
77
77
77
77
78
78
78
78
78
78
79
79
79
79
79
79
79
79
79
80
80
80
80
80
80
80
80
80
81
82
82
82
83
83
83
83
83
83
83
83
83
83
84
84
84
84
84
84
84
84
84
85
85
85
85
85
85
85
85
85
85
85
85
86
86
87
87
87
87
87
87
87
87
87
87
88
88
89
89
89
89
89
89
89
89
89
90
90
90
90
91
91
91
91
91
92
92
92
92
92
92
92
93
93
93
93
93
93
93
93
94
94
94
94
94
94
94
94
94
94
95
95
95
95
95
95
95
95
95
95
95
95
95
95
97
97
97
97
98
98
98
98
98
98
98
98
98
98
98
99
99
99
99
99
99
100
100
100
100
100
100
100
101
101
101
101
101
101
101
102
102
102
102
102
102
102
102
102
103
103
103
103
103
103
103
103
104
104
104
104
104
105
105
105
105
105
105
105
105
106
106
106
106
106
106
106
106
106
106
106
107
107
107
107
107
107
107
107
107
108
108
108
108
109
109
109
109
109
109
109
109
109
109
109
110
110
110
110
111
111
111
111
1

In [5]:
# Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test(data_path=data_path, data_arrays_path=data_arrays_path, split_ratio=0.8)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (3276, 128, 597)
X_test:  (819, 128, 597)


In [6]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')

# reshaping the data for oversampling
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])

# fit and apply the transform
X_train, y_train = oversample.fit_resample(X_train, y_train)
X_test, y_test = oversample.fit_resample(X_test, y_test)

# converting it back to original shape
X_train = X_train.reshape(X_train.shape[0], n_mfcc, max_len_nfft)
X_test = X_test.reshape(X_test.shape[0], n_mfcc, max_len_nfft)

print("X_train: ", X_train.shape, end='\t')
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape, end='\t')
print("y_test: ", y_test.shape)

X_train:  (3348, 128, 597)	y_train:  (3348,)
X_test:  (860, 128, 597)	y_test:  (860,)


In [8]:
# Feature dimension
epochs = 50
batch_size = 64
num_classes = 8

# model
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(n_mfcc, max_len_nfft, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 127, 596, 16)      80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 298, 16)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 298, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 297, 32)       2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 148, 32)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 31, 148, 32)      

In [9]:
model.compile(loss="SparseCategoricalCrossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [11]:
# model.fit(X_train, y_train_hot, epochs=epochs, validation_data=(X_test, y_test_hot))
model.fit(X_train, y_train, epochs=100, batch_size=batch_size, validation_data=(X_test, y_test), shuffle=True)

Epoch 1/100
53/53 [==============================] - 8s 143ms/step - loss: 0.7949 - accuracy: 0.7133 - val_loss: 0.7946 - val_accuracy: 0.7116
Epoch 2/100
53/53 [==============================] - 8s 144ms/step - loss: 0.7874 - accuracy: 0.7243 - val_loss: 0.8022 - val_accuracy: 0.7070
Epoch 3/100
53/53 [==============================] - 8s 144ms/step - loss: 0.8014 - accuracy: 0.7031 - val_loss: 0.7937 - val_accuracy: 0.7198
Epoch 4/100
53/53 [==============================] - 8s 144ms/step - loss: 0.7725 - accuracy: 0.7201 - val_loss: 0.7202 - val_accuracy: 0.7593
Epoch 5/100
53/53 [==============================] - 8s 145ms/step - loss: 0.7315 - accuracy: 0.7363 - val_loss: 0.7202 - val_accuracy: 0.7535
Epoch 6/100
53/53 [==============================] - 8s 144ms/step - loss: 0.7307 - accuracy: 0.7360 - val_loss: 0.7246 - val_accuracy: 0.7651
Epoch 7/100
53/53 [==============================] - 8s 144ms/step - loss: 0.7122 - accuracy: 0.7419 - val_loss: 0.7106 - val_accuracy: 0.7616

In [12]:
model.save("./drive/MyDrive/Colab Notebooks/Models/Room Detection Model")

INFO:tensorflow:Assets written to: ./drive/MyDrive/Colab Notebooks/Models/Room Detection Model/assets


In [ ]:
mfcc_vectors = []
wavfiles = [data_path + "Bathroom" + '/' + wavfile for wavfile in os.listdir(data_path + '/' + "Bathroom")]
for wavfile in wavfiles:
    mfcc = wav2mfcc(wavfile, nfft_array=[], max_len=597, n_mfcc=128)
    mfcc_vectors.append(mfcc)

In [ ]:
#Bethaq,Garage,Bathroom,Bedroom,Kitchen Lounge  Washroom
y = []
j = 1
for i in range(len(mfcc_vectors)):
    test_point = mfcc_vectors[i].reshape(1, 128, 597)
    result = model.predict(test_point)
    result = result.argmax(axis=1)
    print(result)
    if result == 1:
      j += 1
print(j)

[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[6]
[2]
[3]
[2]
[5]
[2]
[2]
[2]
[2]
[6]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[4]
[0]
[2]
[2]
[2]
[2]
[2]
[2]
[6]
[4]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[4]
[2]
[0]
[2]
[2]
[2]
[4]
[0]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[6]
[2]


In [ ]:
#Bethaq,Garage,Bathroom,Bedroom,Kitchen Lounge  Washroom
test_data = wav2mfcc("/content/Ruha lounge ki light off karo.wav", [], 128,597 )
test_data = test_data.reshape(1, 128, 597)
result = model.predict(test_data)
print(result)
result = result.argmax(axis=1)
print(result)

[[1.0174510e-06 1.8946998e-02 2.7674983e-05 5.9212139e-06 2.2094098e-06
  9.8066390e-01 3.5218400e-04]]
[5]
